In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# params we will probably want to do some hyperparameter optimization later
BASE_MODEL= 'ResNet101' # ['VGG16', 'RESNET52', 'InceptionV3', 'Xception', 'DenseNet169', 'DenseNet121']
IMG_SIZE = (224, 224) # [(224, 224), (384, 384), (512, 512), (640, 640)]
BATCH_SIZE = 16 # [1, 8, 16, 24]
DENSE_COUNT = 128 # [32, 64, 128, 256]
DROPOUT = 0.25 # [0, 0.25, 0.5]
LEARN_RATE = 1e-4 # [1e-4, 1e-3, 4e-3]
TRAIN_SAMPLES = 8000 # [3000, 6000, 15000]
TEST_SAMPLES = 800
USE_ATTN = False # [True, False]

In [ ]:
import pandas as pd
male_adult_df = pd.read_csv('/content/drive/MyDrive/Pneumonia/separated rsna overview/male_df.csv')
female_adult_df = pd.read_csv('/content/drive/MyDrive/Pneumonia/separated rsna overview/female_df.csv')
child_df = pd.read_csv('/content/drive/MyDrive/Pneumonia/separated rsna overview/child_df.csv')
male_pa_df = pd.read_csv('/content/drive/MyDrive/Pneumonia/male_adult_pa_pneumonea/male_adult_pa.csv')
female_pa_df = pd.read_csv('/content/drive/MyDrive/Pneumonia/female_adult_pa_pneumonia/female_pa_adult.csv')


In [ ]:
#new column, male  = 0, female  = 1
male_pa_df['gender'] = 0
female_pa_df['gender'] = 1
mf_df = pd.concat([male_pa_df,female_pa_df], ignore_index = True)
mf_df.head()

In [ ]:
mf_df.iloc[0, 5]

In [ ]:
mf_df.groupby('gender').size()

In [ ]:
working_df = mf_df.copy() #male_pa_df.copy() # female_pa_df.copy() #mf_df.copy() # #

In [ ]:
working_df.head()

In [ ]:
working_df = working_df.drop(working_df[working_df['class'] == 'No Lung Opacity / Not Normal'].index)
working_df['new_class'] = working_df['class'].map({ 'Normal':'Non-Pneumonia','Lung Opacity': 'Pneumonia' })

In [ ]:
working_df = working_df.drop('Unnamed: 0', axis = 1)
#working_df['new_class'] = working_df['class'].map({'No Lung Opacity / Not Normal' : 'Non-Pneumonia', 'Normal':'Non-Pneumonia','Lung Opacity': 'Pneumonia' })

In [ ]:
working_df['path'] = working_df['path'].str.replace('../input/rsna-pneumonia-detection-challenge','/content/drive/MyDrive/Pneumonia/rsna dataset')

In [ ]:
working_df.iloc[0,4]

In [ ]:
working_df.groupby('class').size()

In [ ]:
# get the labels in the right format
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
class_enc = LabelEncoder()
working_df['class_idx'] = class_enc.fit_transform(working_df['new_class'])
oh_enc = OneHotEncoder(sparse=False)
working_df['class_vec'] = oh_enc.fit_transform(
    working_df['class_idx'].values.reshape(-1, 1)).tolist() 
working_df.sample(3)

In [ ]:
from sklearn.model_selection import train_test_split
image_df = working_df.groupby('patientId').apply(lambda x: x.sample(1))
raw_train_df, valid_df = train_test_split(image_df, test_size=0.20, random_state=2018,
                                    stratify=image_df['new_class'])
print(raw_train_df.shape, 'training data')
print(valid_df.shape, 'validation data')

In [ ]:
raw_train_df.groupby('class').size()

In [ ]:
raw_train_df.groupby('new_class').size()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))
raw_train_df.groupby('new_class').size().plot.bar(ax=ax1)
#train_df = raw_train_df.groupby('new_class').\
#   apply(lambda x: x.sample(12000//3)).\
#   reset_index(drop=True)
train_df = raw_train_df.groupby('class').apply(lambda x: x.sample(n={'Lung Opacity':1076, 'Normal':1076}.get(x.name))).reset_index(drop = True)

train_df.groupby('new_class').size().plot.bar(ax=ax2) 
#train_df.groupby('class').size().plot.bar() 
print(train_df.shape[0], 'new training size')

In [ ]:
pip install pydicom

In [ ]:
try:
    # keras 2.2
    import keras_preprocessing.image.utils as KPImageUtils
    import keras_preprocessing.image as KPImage
except:
    # keras 2.1
    import keras.preprocessing.image as KPImage
    
from PIL import Image
import pydicom
def read_dicom_image(in_path):
    img_arr = pydicom.read_file(in_path).pixel_array
    return img_arr/img_arr.max()
    
class medical_pil():
    @staticmethod
    def open(in_path):
        if '.dcm' in in_path:
            c_slice = read_dicom_image(in_path)
            int_slice =  (255*c_slice).clip(0, 255).astype(np.uint8) # 8bit images are more friendly
            return Image.fromarray(int_slice)
        else:
            return Image.open(in_path)
    fromarray = Image.fromarray

In [ ]:
KPImageUtils.pil_image = medical_pil

In [ ]:
from PIL import Image
KPImageUtils._PIL_INTERPOLATION_METHODS = {
        'nearest': Image.NEAREST,
        'bilinear': Image.BILINEAR,
        'bicubic': Image.BICUBIC,
    }

def load_img(path, grayscale=False, color_mode='rgb', target_size=None,
                 interpolation='nearest'):
        """Loads an image into PIL format.
        # Arguments
            path: Path to image file.
            grayscale: DEPRECATED use `color_mode="grayscale"`.
            color_mode: The desired image format. One of "grayscale", "rgb", "rgba".
                "grayscale" supports 8-bit images and 32-bit signed integer images.
                Default: "rgb".
            target_size: Either `None` (default to original size)
                or tuple of ints `(img_height, img_width)`.
            interpolation: Interpolation method used to resample the image if the
                target size is different from that of the loaded image.
                Supported methods are "nearest", "bilinear", and "bicubic".
                If PIL version 1.1.3 or newer is installed, "lanczos" is also
                supported. If PIL version 3.4.0 or newer is installed, "box" and
                "hamming" are also supported.
                Default: "nearest".
        # Returns
            A PIL Image instance.
        # Raises
            ImportError: if PIL is not available.
            ValueError: if interpolation method is not supported.
        """
        if grayscale is True:
            warnings.warn('grayscale is deprecated. Please use '
                          'color_mode = "grayscale"')
            color_mode = 'grayscale'
        if KPImageUtils.pil_image is None:
            raise ImportError('Could not import PIL.Image. '
                              'The use of `load_img` requires PIL.')
        with open(path, 'rb') as f:
            img = KPImageUtils.pil_image.open(path)
            if color_mode == 'grayscale':
                # if image is not already an 8-bit, 16-bit or 32-bit grayscale image
                # convert it to an 8-bit grayscale image.
                if img.mode not in ('L', 'I;16', 'I'):
                    img = img.convert('L')
            elif color_mode == 'rgba':
                if img.mode != 'RGBA':
                    img = img.convert('RGBA')
            elif color_mode == 'rgb':
                if img.mode != 'RGB':
                    img = img.convert('RGB')
            else:
                raise ValueError('color_mode must be "grayscale", "rgb", or "rgba"')
            if target_size is not None:
                width_height_tuple = (target_size[1], target_size[0])
                if img.size != width_height_tuple:
                    if interpolation not in KPImageUtils._PIL_INTERPOLATION_METHODS:
                        raise ValueError(
                            'Invalid interpolation method {} specified. Supported '
                            'methods are {}'.format(
                                interpolation,
                                ", ".join(KPImageUtils._PIL_INTERPOLATION_METHODS.keys())))
                    resample = KPImageUtils._PIL_INTERPOLATION_METHODS[interpolation]
                    img = img.resize(width_height_tuple, resample)
            return img

        
KPImageUtils.load_img = load_img        
def _get_batches_of_transformed_samples(self, index_array):
    """Gets a batch of transformed samples.
    # Arguments
        index_array: Array of sample indices to include in batch.
    # Returns
        A batch of transformed samples.
    """
    #for i, n_observation in enumerate(index_array):
     #   print(str(i) + "+ " + str( n_observation))
    batch_x = np.zeros((len(index_array),) + self.image_shape, dtype=self.dtype)
    # build batch of image data
    # self.filepaths is dynamic, is better to call it once outside the loop
    filepaths = self.filepaths
    for i, j in enumerate(index_array):
        img = KPImageUtils.load_img(filepaths[j],
                       color_mode=self.color_mode,
                       target_size=self.target_size,
                       interpolation=self.interpolation)
        x = KPImageUtils.img_to_array(img, data_format=self.data_format)
        # Pillow images should be closed after `load_img`,
        # but not PIL images.
        if hasattr(img, 'close'):
            img.close()
        if self.image_data_generator:
            params = self.image_data_generator.get_random_transform(x.shape)
            x = self.image_data_generator.apply_transform(x, params)
            x = self.image_data_generator.standardize(x)
        batch_x[i] = x
    # optionally save augmented images to disk for debugging purposes
    if self.save_to_dir:
        for i, j in enumerate(index_array):
            img = KPImageUtils.array_to_img(batch_x[i], self.data_format, scale=True)
            fname = '{prefix}_{index}_{hash}.{format}'.format(
                prefix=self.save_prefix,
                index=j,
                hash=np.random.randint(1e7),
                format=self.save_format)
            img.save(os.path.join(self.save_to_dir, fname))
    # build batch of labels
    if self.class_mode == 'input':
        batch_y = batch_x.copy()
    elif self.class_mode in {'binary', 'sparse'}:
        #print(str(self.classes.shape))
        #print(str(len(batch_x)))
        try:
            batch_y = np.empty([len(batch_x), self.classes.shape[1]], dtype=self.dtype)
        except:
            batch_y = np.empty([len(batch_x), 1], dtype=self.dtype)
        #print(batch_y)
        
        #print(str(len(index_array)))
        
        #print(str(batch_y.shape))
        
        for i, n_observation in enumerate(index_array):
            #print(self.classes[n_observation])
            #print(batch_y[i])
            batch_y[i] = self.classes[n_observation]
    elif self.class_mode == 'categorical':
        batch_y = np.zeros((len(batch_x), len(self.class_indices)),
                           dtype=self.dtype)
        for i, n_observation in enumerate(index_array):
            batch_y[i, self.classes[n_observation]] = 1.
    elif self.class_mode == 'multi_output':
        batch_y = [output[index_array] for output in self.labels]
    elif self.class_mode == 'raw':
        batch_y = self.labels[index_array]
    else:
        return batch_x
    if self.sample_weight is None:
        return batch_x, batch_y
    else:
        return batch_x, batch_y, self.sample_weight[index_array]

In [ ]:
from keras_preprocessing.image.iterator import BatchFromFilesMixin
BatchFromFilesMixin._get_batches_of_transformed_samples = _get_batches_of_transformed_samples

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='RESNET52':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet50':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet101':
    from keras.applications.resnet import ResNet101 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet201':
    from keras.applications import DenseNet201 as PTModel
    from keras.applications.densenet import  preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))

Image Augmentation

In [ ]:
img_gen_args = dict(samplewise_center=False, 
                              samplewise_std_normalization=False, 
                              horizontal_flip = False, 
                              vertical_flip = False, 
                              height_shift_range = 0.05, 
                              width_shift_range = 0.02, 
                              rotation_range = 3, 
                              shear_range = 0.01,
                              fill_mode = 'nearest',
                              zoom_range = 0.05,
                              preprocessing_function=preprocess_input,) # preprocessing_function=preprocess_input ###from vgg16 inplace of rescale 
img_gen = ImageDataGenerator(**img_gen_args)

In [ ]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, seed = None, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways: seed: {}'.format(seed))
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'binary',
                                              seed = seed,
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values,0)
    df_gen.filepaths.extend(df_gen.filenames)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

In [ ]:
train_gen = flow_from_dataframe(img_gen, train_df, #raw train df if use unbalance data
                             path_col = 'path',
                            y_col = 'class_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = BATCH_SIZE)

valid_gen = flow_from_dataframe(img_gen, valid_df, 
                             path_col = 'path',
                            y_col = 'class_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = BATCH_SIZE) # we can use much larger batches for evaluation 256
# used a fixed dataset for evaluating the algorithm
valid_X, valid_Y = next(flow_from_dataframe(img_gen, 
                               valid_df, 
                             path_col = 'path',
                            y_col = 'class_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb', #rgb
                            batch_size = TEST_SAMPLES)) # one big batch

In [ ]:
t_x, t_y = next(train_gen)
print(t_x.shape, t_y.shape)

In [ ]:
train_gen.classes

In [ ]:
!python --version

Model architecture

In [ ]:
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda, AvgPool2D, LeakyReLU
from keras.models import Model
from keras.optimizers import Adam
from keras import regularizers
from keras.models import Sequential
from keras.optimizers import Adam,  RMSprop
#with strategy.scope():
base_pretrained_model = PTModel(input_shape =  t_x.shape[1:], 
                          include_top = False, weights = 'imagenet')
base_pretrained_model.trainable = False



In [ ]:
base_pretrained_model.summary()

In [ ]:

pt_features = Input(base_pretrained_model.layers[-1].output.shape[1:], name = 'feature_input')
pt_depth = base_pretrained_model.layers[-1].output.shape[1:]

from keras.layers import BatchNormalization

bn_features = BatchNormalization()(pt_features)
gap = GlobalAveragePooling2D()(bn_features)

gap_dr = Dropout(DROPOUT)(gap)
dr_steps = Dropout(DROPOUT)(Dense(DENSE_COUNT, activation = 'elu')(gap_dr))

#dr_steps = BatchNormalization()(dr_steps)
#dr_steps = LeakyReLU(0.1)(dr_steps)

out_layer = Dense(t_y.shape[1], activation = 'softmax')(dr_steps)

attn_model = Model(inputs = [pt_features], 
                   outputs = [out_layer], name = 'trained_model')
pneu_model = Sequential(name = 'combined_model')
base_pretrained_model.trainable = False
pneu_model.add(base_pretrained_model)
pneu_model.add(attn_model)
pneu_model.compile(optimizer = RMSprop(lr=0.0001, decay=1e-5), loss = 'binary_crossentropy', #Adam(lr = LEARN_RATE)
                           metrics = ['binary_accuracy'])
pneu_model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('lung_opacity')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, 
                                   patience=10, verbose=1, mode='auto', 
                                   min_delta=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=10) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
train_gen.batch_size = BATCH_SIZE
history = pneu_model.fit_generator(train_gen, 
                         steps_per_epoch=train_gen.n//BATCH_SIZE,
                         validation_data=(valid_X, valid_Y), 
                         epochs=10, 
                         callbacks=callbacks_list,
                         workers=2)

In [ ]:


pred_Y = pneu_model.predict(valid_X, 
                          batch_size = BATCH_SIZE,  
                          verbose = True) 



In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
plt.matshow(confusion_matrix(np.argmax(valid_Y, -1), np.argmax(pred_Y,-1)))
print(classification_report(np.argmax(valid_Y, -1), 
                            np.argmax(pred_Y,-1), target_names = class_enc.classes_))

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, _ = roc_curve(np.argmax(valid_Y,-1)==0, pred_Y[:,0])
fig, ax1 = plt.subplots(1,1, figsize = (5, 5), dpi = 250)
ax1.plot(fpr, tpr, 'b.-', label = 'Densenet121 (AUC:%2.2f)' % roc_auc_score(np.argmax(valid_Y,-1)==0, pred_Y[:,0]))
ax1.plot(fpr, fpr, 'k-', label = 'Random Guessing')
ax1.legend(loc = 4)
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate');
ax1.set_title('Lung Opacity ROC Curve')
fig.savefig('roc_valid.pdf') 

Load local test data

In [ ]:
local_output = pd.read_csv('/content/drive/MyDrive/Pneumonia/loca_data_output_final/local_data_output_final.csv')
local_output = local_output.drop(['Unnamed: 0'], axis = 1)

In [ ]:
local_output['Doctor 1/Ground Truth'] = local_output['Doctor 1/Ground Truth'].map({'No':0, 'Yes': 1})
local_output['Doctor 2'] = local_output['Doctor 2'].map({'No':0, 'Yes': 1})
local_output['Doctor 3'] = local_output['Doctor 3'].map({'No':0, 'Yes': 1})
local_output['class'] = local_output['class'].map({'No':0, 'Yes': 1})
local_output.head()

In [ ]:


TEST_DIR2 =  "/content/drive/MyDrive/Pneumonia/local separated images"
batch_size_test = 1
nb_test_samples2 = 129

test_generator2 = img_gen.flow_from_directory(
    TEST_DIR2,
    classes=['male-female-mod'],
    # don't generate labels
    class_mode='binary',
    # don't shuffle
    shuffle=False,
    target_size = IMG_SIZE,
    color_mode = 'rgb',
    batch_size=batch_size_test,)



In [ ]:
test_generator2.reset()
preds2 = pneu_model.predict_generator(test_generator2, workers = 1,steps=np.ceil(nb_test_samples2/batch_size_test)) #changed worker to 0

In [ ]:


preds2 = np.argmax(preds2, axis=-1)
data = {'col_1': test_generator2.filenames, 'col_2': preds2}
df=pd.DataFrame.from_dict(data)



In [ ]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
ids = [x for x in df['col_1'] ]
ids = [natural_keys(x)[1] for x in ids]
df['ID'] = np.asarray(ids)
df = df.sort_values(by='ID')
df = df.reset_index(drop  =True)

In [ ]:
df

In [ ]:
df[df['col_2'] == 1]

In [ ]:
df[df['col_2'] == 1].shape

In [ ]:
plt.matshow(confusion_matrix(np.asarray(local_output['class']), np.asarray(df['col_2'])))
print(classification_report(np.asarray(local_output['class']), 
                           np.asarray(df['col_2']), target_names = class_enc.classes_))

In [ ]:


from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, _ = roc_curve(np.asarray(local_output['class'])==0, np.asarray(df['col_2'] == 0))
fig, ax1 = plt.subplots(1,1, figsize = (5, 5), dpi = 250)
ax1.plot(fpr, tpr, 'b.-', label = 'Densenet121 (AUC:%2.2f)' % roc_auc_score(np.asarray(local_output['class'])==0,np.asarray(df['col_2'] == 0)))
ax1.plot(fpr, fpr, 'k-', label = 'Random Guessing')
ax1.legend(loc = 4)
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate');
ax1.set_title('Lung Opacity ROC Curve')
fig.savefig('roc_valid.pdf') 



In [ ]:
pneu_model.save('resnet101.h5')

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (224, 224, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

Prediction for each image

In [ ]:
ids = [i for i in df['ID']]
preds = []
for i in ids:
  image = load('/content/drive/MyDrive/Pneumonia/local separated images/male-female-mod/' + str(i) + '.jpg')
  pred = pneu_model.predict(image)
  preds.append(pred)



In [ ]:
data = {'image_id':ids , 'probability(no and yes)': preds}
df=pd.DataFrame.from_dict(data)

In [ ]:
df

In [ ]:
df.to_csv('male female pa resnet101.csv')